In [ ]:
#import modules
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

from keras.constraints import NonNeg,UnitNorm
from keras import regularizers
from keras.optimizers import Adam
import tensorflow as tf

import scipy.ndimage

In [ ]:
# To otain same results
from numpy.random import seed
seed(1)
#from tensorflow import set_random_seed
#set_random_seed(2)
tf.random.set_seed(2)

In [ ]:
# load pixel values 

#change accordingly (jsaper:4,198,100 samson:3,156,95)
r = 4  # end members
n = 198 # spectral bands
size = 100 # window size

Y = np.loadtxt('jasper.csv', delimiter=',')
Y = np.reshape(Y, (n, size, size))
x_train = Y.astype('float32') 

x_train = np.reshape(x_train, (n, size, size, 1))  # channel last
y_train = np.reshape(x_train, (n, size*size))


#add noise for SNR test
#g_noise = np.random.normal(0,0.01,(n, size, size, 1))
#x_train = x_train + g_noise



In [ ]:
#model
input_img = Input(shape=(size, size, 1)) 

#Encoder cnn part
x = Conv2D(16, (3, 3), activation='relu', padding='same', name='Conv1')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same', name='Conv2')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same', name='Conv3')(x)
x = MaxPooling2D((2, 2), padding='same')(x)


# Encoder FC part
x = Flatten()(x)
x = Dense(9*r, activation='relu')(x)
#x = Dropout(0.01)(x)
x = Dense(6*r, activation='relu')(x)
#x = Dropout(0.01)(x)
x = Dense(3*r, activation='relu')(x)

# Decoder
x = Dropout(0.01)(x)
x = Dense(r)(x)
#x = BatchNormalization()(x)
x = Activation('relu', name='Activation')(x)
decoded = Dense(size*size, activation='linear', use_bias=False, trainable=True, kernel_constraint=NonNeg(), kernel_regularizer=regularizers.l2(0.0001))(x)


autoencoder = Model(input_img, decoded)

#custom loss function
#def custom_loss(y_true, y_pred):
#    y_true = K.clip(y_true, K.epsilon(), 1)
#    y_pred = K.clip(y_pred, K.epsilon(), 1)
#    return K.sum(y_true * (K.log(1/y_pred)), axis=-1)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')
#autoencoder.compile(optimizer='adam', loss=custom_loss)
autoencoder.summary()

In [ ]:
#train the model
autoencoder.fit(x_train, y_train, epochs=100)

In [ ]:
# get S (abundance) by weights
W = autoencoder.get_weights()
S = W[-1][:] # need to change this with number of layers. 

# get A (end members) by activations
intermediate_layer_model = Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('Activation').output) #change layer name
A = intermediate_layer_model.predict(x_train)


In [ ]:
#visualize the output
decoded_imgs = autoencoder.predict(x_train).reshape(n,size,size)
decoded_imgs.shape

n = 10
s = 20   # 15 if samson 
plt.figure(figsize=(40, 8))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_train[i*s].reshape(size, size))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n+1)
    plt.imshow(decoded_imgs[i*s].reshape(size, size))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
#save to csv
np.savetxt('A_pred.csv', A, delimiter=",") # End Members
np.savetxt('S_pred.csv', S, delimiter=",") # Abundances

In [ ]:
#Visualize the intermeidiate layer outputs for a specific band 
band = 2
band = band-1

# Conv1
intermediate_layer_conv1 = Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('Conv1').output) #change layer name
c1 = intermediate_layer_conv1.predict(x_train)
for i in range(16):
    #np.savetxt('img\decode'+str(i+1)+'.csv',decoded_imgs[i,:,:], delimiter=",")
    plt.imsave('img\conv1\c1_'+str(i+1)+'.png',c1[band,:,:,i])

# Conv2
intermediate_layer_conv2 = Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('Conv2').output) #change layer name
c2 = intermediate_layer_conv2.predict(x_train)
for i in range(8):
    #np.savetxt('img\decode'+str(i+1)+'.csv',decoded_imgs[i,:,:], delimiter=",")
    c2_rescaled = scipy.ndimage.zoom(c2[band ,:,:,i], 2, order=1)
    plt.imsave('img\conv2\c2_'+str(i+1)+'.png',c2_rescaled)
    
# Conv3
intermediate_layer_conv3 = Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('Conv3').output) #change layer name
c3 = intermediate_layer_conv3.predict(x_train)
for i in range(8):
    #np.savetxt('img\decode'+str(i+1)+'.csv',decoded_imgs[i,:,:], delimiter=",")
    c3_rescaled = scipy.ndimage.zoom(c3[band ,:,:,i], 4, order=1)
    plt.imsave('img\conv3\c3_'+str(i+1)+'.png',c3_rescaled)